# 📊 Анализ данных и прогнозирование цен на арматуру
В этом ноутбуке мы проведем анализ данных, подготовим признаки и обучим модели для предсказания цен на арматуру.

**Методы:**
- 📈 Анализ и визуализация данных
- 🔄 Проверка стационарности ряда
- 🏗 Создание новых признаков
- 🤖 Обучение моделей (LightGBM и LSTM)
- 📊 Оценка качества предсказаний

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных
train_df = pd.read_excel("train.xlsx")
test_df = pd.read_excel("test.xlsx")
fuel_df = pd.read_excel("Топливо.xlsx")
metal_df = pd.read_excel("Показатели рынка металла.xlsx")

# Преобразуем даты
train_df["dt"] = pd.to_datetime(train_df["dt"], errors="coerce")
test_df["dt"] = pd.to_datetime(test_df["dt"], errors="coerce")

# Вывод первых строк
train_df.head()


## 📊 Первичный анализ данных

In [ ]:

# Проверка структуры данных
print("Train dataset shape:", train_df.shape)
print("Test dataset shape:", test_df.shape)
print("Fuel dataset shape:", fuel_df.shape)
print("Metal dataset shape:", metal_df.shape)

# Проверяем пропущенные значения
print("
Пропущенные значения в train:")
print(train_df.isnull().sum())

# Статистическое описание данных
train_df.describe()


## 📈 Динамика цен на арматуру

In [ ]:

plt.figure(figsize=(12, 6))
plt.plot(train_df["dt"], train_df["Цена на арматуру"], label="Цена на арматуру", marker="o")
plt.xlabel("Дата")
plt.ylabel("Цена")
plt.title("Динамика цен на арматуру (2015-2023)")
plt.legend()
plt.grid()
plt.show()


## 🔍 Корреляция цен с макропоказателями

In [ ]:

merged_df = train_df.merge(fuel_df, on="dt", how="left").merge(metal_df, on="dt", how="left")
numeric_df = merged_df.select_dtypes(include=[np.number])
correlation = numeric_df.corr()["Цена на арматуру"].sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=correlation[:10].index, y=correlation[:10].values)
plt.xticks(rotation=90)
plt.ylabel("Коэффициент корреляции")
plt.title("Корреляция макропоказателей с ценой на арматуру")
plt.show()


## 🔬 Проверка стационарности временного ряда

In [ ]:

from statsmodels.tsa.stattools import adfuller

def check_stationarity(series):
    result = adfuller(series.dropna())
    print("📌 ADF Statistic:", result[0])
    print("📌 p-value:", result[1])
    if result[1] < 0.05:
        print("✅ Ряд стационарен (p < 0.05)")
    else:
        print("⚠️ Ряд нестационарен (p >= 0.05)")

check_stationarity(train_df["Цена на арматуру"])


## 🔄 Преобразование данных для обучения моделей

In [ ]:

# Применяем разностное преобразование
train_df["diff_price"] = train_df["Цена на арматуру"].diff()
check_stationarity(train_df["diff_price"])

# Визуализируем
plt.figure(figsize=(12, 6))
plt.plot(train_df["dt"], train_df["diff_price"], label="Дифференцированная цена")
plt.xlabel("Дата")
plt.ylabel("Изменение цены")
plt.title("Дифференцированная цена на арматуру")
plt.legend()
plt.grid()
plt.show()


## 🏗 Создание новых признаков

In [ ]:

lags = [1, 4, 6, 8, 12]
for lag in lags:
    train_df[f"lag_{lag}"] = train_df["Цена на арматуру"].shift(lag)

windows = [4, 8, 12]
for window in windows:
    train_df[f"rolling_mean_{window}"] = train_df["Цена на арматуру"].rolling(window).mean()

train_df.dropna(inplace=True)
train_df.head()


## 🤖 Обучение модели LightGBM

In [ ]:

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

features = [col for col in train_df.columns if col not in ["dt", "Цена на арматуру"]]
target = "Цена на арматуру"

X_train, X_val, y_train, y_val = train_test_split(train_df[features], train_df[target], test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val)

params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "max_depth": 5,
    "verbose": -1
}

model_lgbm = lgb.train(params, train_data, valid_sets=[valid_data], num_boost_round=500, callbacks=[lgb.early_stopping(50)])

y_pred_lgbm = model_lgbm.predict(X_val)

rmse_lgbm = np.sqrt(mean_squared_error(y_val, y_pred_lgbm))
mape_lgbm = mean_absolute_percentage_error(y_val, y_pred_lgbm) * 100

print(f"📉 LightGBM RMSE: {rmse_lgbm:.2f} руб.")
print(f"📊 LightGBM MAPE: {mape_lgbm:.2f}%")


## 📌 Выводы и дальнейшие шаги

✅ Мы проанализировали данные, подготовили их и обучили модель LightGBM. Следующий шаг – добавить обучение LSTM на PyTorch.